In [1]:
import json
import pandas as pd

In [15]:
ls data

dailystock-pruned.csv  options-better.csv     options_brief.csv
dailystock.csv         options-with-date.csv  treasury-better.csv
date-to-closing.json   options.csv            treasury.csv


In [12]:
treasury = pd.read_csv('data/treasury-better.csv')
with open('data/date-to-closing.json') as f:
    date_to_closing = {int(k): v for k, v in json.load(f).items()}

In [17]:
options = pd.read_csv('data/options-with-date.csv')

In [27]:
dates_in_options = set(options.date.values)

In [29]:
assert not dates_in_options - set(date_to_closing.keys())

In [32]:
not_in_treasury = dates_in_options - set(treasury.date.values)

In [54]:
date_to_treasury_vals = {}
for idx, row in treasury.iterrows():
    d = dict(row)
    date_to_treasury_vals[int(d['date'])] = d
    del date_to_treasury_vals[int(d['date'])]['date']

In [56]:
# date_to_treasury_vals

In [43]:
d = dict(options.date.value_counts())
sum(d[x] for x in not_in_treasury)

96613

In [59]:
treasury_dates = set(treasury.date.values)
modified_options = options[options.date.isin(treasury_dates)]

In [87]:
def get_treasury_rate(row):
    date = row.date
    dur = row.date_ndiff
    if dur <= 45:
        select = '1MO'
    elif dur <= 75:
        select = '2MO'
    elif dur <= 135:
        select = '3MO'
    elif dur <= 270:
        select = '6MO'
    elif dur <= 547:
        select = '1YR'
    elif dur <= 912:
        select = '2YR'
    elif dur <= 1460:
        select = '3YR'
    elif dur <= 2190:
        select = '5YR'
    elif dur <= 3102:
        select = '7YR'
    elif dur <= 3975:
        select = '10YR'
    elif dur <= 6625:
        select = '20YR'
    else:
        select = '30YR'
    curr = date_to_treasury_vals[date][select]
    return curr if curr >= 0 else date_to_treasury_vals[date]['3MO']

In [88]:
modified_options['treasury_rate'] = modified_options.apply(lambda row: get_treasury_rate(row), axis=1)

/Users/ycm/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [96]:
modified_options['closing_price'] = modified_options.apply(lambda row: date_to_closing[row.date], axis=1)

/Users/ycm/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [98]:
modified_options.to_csv('data/options-df.csv', index=None)